In [1]:
from confluent_kafka import Producer
import json
from datetime import datetime, timedelta
# ... Phần code lấy dữ liệu từ Finviz ...
import requests
import time
def get_finviz_data(ticker, timeframe):
    # Xác định thời điểm 6 tiếng trước từ thời điểm hiện tại
    six_hours_ago = datetime.utcnow() - timedelta(hours=1)

    # Chuyển đổi thời điểm thành Unix timestamp
    timestamp_six_hours_ago = int(six_hours_ago.timestamp())

    url = "https://finviz.com/api/quote.ashx"
    params = {
        'aftermarket': 'false',
        'instrument': 'forex',
        'patterns': 'true',
        'premarket': 'false',
        'rev': "1702484426227",  # Sử dụng timestamp_six_hours_ago thay vì một giá trị cứng
        'ticker': ticker,
        'timeframe': timeframe,
        'type': 'new',
        'format': 'json'
    }

    # Sử dụng UserAgent để giả mạo trình duyệt
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    # Thực hiện yêu cầu API với headers giả mạo
    response = requests.get(url, params=params, headers=headers)

    # Kiểm tra xem yêu cầu có thành công không (status code 200 là thành công)
    if response.status_code == 200:
        # Chuyển đổi dữ liệu JSON thành đối tượng Python
        data = json.loads(response.text)
        # Giữ lại 10 phần tử cuối cùng của mỗi mảng
        data['volume'] =  data['volume'][-10:]
        data['date'] =  data['date'][-10:]
        data['date'] = [(datetime.utcfromtimestamp(ts) + timedelta(hours=7)).strftime('%Y%m%d%H%M') for ts in data['date']]
        # data['open'] =  data['open'][-10:]
        data['high'] =  data['high'][-10:]
        data['low'] =  data['low'][-10:]
        data['close'] =  data['close'][-10:]
        return data
    else:
        print(f"Yêu cầu API không thành công. Mã trạng thái: {response.status_code}")
        print(response.text)
        return None
    
def send_data_to_kafka(producer, topic, data):
    # producer.produce(topic, value=json.dumps(data).encode('utf-8'))
    producer.produce(topic, value="value")
    producer.flush()

def main():
    # ... Các thiết lập cho Kafka ...

    ticker_input = 'USDCHF'
    timeframe_input = 'i30'
    producer = Producer({'bootstrap.servers': 'localhost:9092'})
    topic = "USDCHF_i30"

    while True:
        result = get_finviz_data(ticker_input, timeframe_input)
        print("Data:", result)

        if result:
            send_data_to_kafka(producer, topic, result)
            print("Data sent to Kafka:", result)
        else:
            print("No data to send")

        # Thay đổi thời gian nghỉ giữa các lần gửi dữ liệu nếu cần
        time.sleep(5)

if __name__ == "__main__":
    main()


Data: {'ticker': 'USDCHF', 'name': 'USD/CHF', 'timeframe': 'i30', 'volume': [5507, 1391, 1831, 2887, 2891, 2766, 3424, 2751, 2188, 128], 'date': ['202312180600', '202312180630', '202312180700', '202312180730', '202312180800', '202312180830', '202312180900', '202312180930', '202312181000', '202312181027'], 'open': [0.88751, 0.88703, 0.88693, 0.88718, 0.88759, 0.88804, 0.88733, 0.88712, 0.88748, 0.88734, 0.88808, 0.88729, 0.88642, 0.88669, 0.88707, 0.88666, 0.88649, 0.88602, 0.88546, 0.88554, 0.88575, 0.88576, 0.88567, 0.88602, 0.88559, 0.88467, 0.88473, 0.88536, 0.88594, 0.88598, 0.88637, 0.88611, 0.88525, 0.88505, 0.88455, 0.88397, 0.88464, 0.88444, 0.88308, 0.88349, 0.88357, 0.88397, 0.884, 0.88421, 0.8848, 0.88444, 0.88421, 0.8845, 0.88348, 0.883, 0.88348, 0.88355, 0.88385, 0.884, 0.88422, 0.88431, 0.8849, 0.88446, 0.88496, 0.8837700000000001, 0.88385, 0.88415, 0.88406, 0.88421, 0.88479, 0.88497, 0.88402, 0.88454, 0.88467, 0.88445, 0.88433, 0.88469, 0.88495, 0.8851, 0.88414, 0.88434,

In [2]:
pip install confluent_kafka

     |████████████████████████████████| 4.0 MB 831 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
